In [1]:
# load the required libraries
from boruta import BorutaPy

ModuleNotFoundError: No module named 'boruta'

##### Package installation

Note: I am on Windows 7 64-bit OS.

1. Open command prompt and type the command `!conda install -c saravji boruta` to install the package.

2. If you get an error like, `could not fetch URL.. problem with SSL certificate` then to solve it, do the following;

- install OpenSSL from [here](https://slproweb.com/products/Win32OpenSSL.html). I chose, `Win64 OpenSSL v1.1.1c Light` version.
- Now open terminal and type the command, ``!conda install -c saravji boruta` to install the package.
- To see list of installed python packages, in terminal type, `pip freeze`.

In [1]:
from boruta import BorutaPy
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier

In [2]:
# print working directory
import os
print(os.getcwd())

C:\Users\Ashoo\Documents\PythonPlayground\AML\scripts\Exploratory Data Analysis (EDA)


In [26]:
# change the path if needed
#%%time
filepath = 'C:/Users/Ashoo/Documents/PythonPlayground/AML/data/adult.csv'
df = pd.read_csv(filepath,dtype={'target': np.int8, 'id': np.int32})
# make a copy of df
df1= df

#### Exploratory Data Analysis

1. Check the number of rows and columns and print column names
2. Print first 5 rows of the dataset
3. Transpose the frame to see all features at once
4. Examine data types of all features and total dataframe size in memory
5. Get basic statistics of each feature
6. Feature Selection (using the Boruta algorithm)

In [17]:
# 1. Check the number of rows and columns and print column names
print(df.shape)
print(df.columns)

(32561, 15)
Index(['age', 'workclass', 'fnlwgt', 'education', 'education-num',
       'marital-status', 'occupation', 'relationship', 'race', 'sex',
       'capital-gain', 'capital-loss', 'hours-per-week', 'native-country',
       'salary'],
      dtype='object')


In [18]:
# 2. Print first 5 rows of the dataset
df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [19]:
# 3. Transpose the frame to see all features at once
df.head().T

,0,1,2,3,4
age,39,50,38,53,28
workclass,State-gov,Self-emp-not-inc,Private,Private,Private
fnlwgt,77516,83311,215646,234721,338409
education,Bachelors,Bachelors,HS-grad,11th,Bachelors
education-num,13,13,9,7,13
marital-status,Never-married,Married-civ-spouse,Divorced,Married-civ-spouse,Married-civ-spouse
occupation,Adm-clerical,Exec-managerial,Handlers-cleaners,Handlers-cleaners,Prof-specialty
relationship,Not-in-family,Husband,Not-in-family,Husband,Wife
race,White,White,White,Black,Black
sex,Male,Male,Male,Male,Female


In [20]:
# 4. Examine data types of all features and total dataframe size in memory
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
age               32561 non-null int64
workclass         32561 non-null object
fnlwgt            32561 non-null int64
education         32561 non-null object
education-num     32561 non-null int64
marital-status    32561 non-null object
occupation        32561 non-null object
relationship      32561 non-null object
race              32561 non-null object
sex               32561 non-null object
capital-gain      32561 non-null int64
capital-loss      32561 non-null int64
hours-per-week    32561 non-null int64
native-country    32561 non-null object
salary            32561 non-null object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB


In [21]:
# 5. Get basic statistics of each feature
df.describe().T

,count,mean,std,min,25%,50%,75%,max
age,32561.0,38.581647,13.640433,17.0,28.0,37.0,48.0,90.0
fnlwgt,32561.0,189778.366512,105549.977697,12285.0,117827.0,178356.0,237051.0,1484705.0
education-num,32561.0,10.080679,2.572720,1.0,9.0,10.0,12.0,16.0
capital-gain,32561.0,1077.648844,7385.292085,0.0,0.0,0.0,0.0,99999.0
capital-loss,32561.0,87.303830,402.960219,0.0,0.0,0.0,0.0,4356.0
hours-per-week,32561.0,40.437456,12.347429,1.0,40.0,40.0,45.0,99.0


##### Label Encoding of the categorical features into numeric

In [23]:
from sklearn.preprocessing import LabelEncoder
label_encoder = preprocessing.LabelEncoder()

In [27]:
# encode all categorical variables
df['workclass']= label_encoder.fit_transform(df['workclass'])
df['education']= label_encoder.fit_transform(df['education'])
df['marital-status']= label_encoder.fit_transform(df['marital-status']) 
df['occupation']= label_encoder.fit_transform(df['occupation']) 
df['relationship']= label_encoder.fit_transform(df['relationship'])
df['race']= label_encoder.fit_transform(df['race'])
df['sex']= label_encoder.fit_transform(df['sex'])
df['native-country']= label_encoder.fit_transform(df['native-country'])
df['salary']= label_encoder.fit_transform(df['salary'])

In [28]:
# encode gender
#df['workclass'].unique()
df['salary'].unique()

array([0, 1], dtype=int64)

### 6. Automated feature selection with Boruta

`Boruta` is an R algorithm, since ported to Python, which implements another approach to feature selection. It's both significantly more complex than the algorithms offered in sklearn, and also carries itself a bit differently in terms of what its goal is. In introducing it I'll pull from the paper that was published alongside the algorithm (Google "Boruta" to find it).

#### Background

###### Minimal-optimal feature selection versus all-relevant feature selection

Basically `Boruta` is an algorithm designed to sovle what the paper calls the "all-relevant problem": finding a subset of features from the dataset which are relevant to given classification task. This is different from the "minimal-optimal problem", which is the problem of finding the minimal subset of features which are performant in a model. While machine learning models in production should ultimately target building on minimal-optimal selections of the data, the thesis of Boruta is that for the purposes of exploration minimum-optimality goes too far.

`Boruta` attempts to curate features down to the "all-relevant" stopping point, not the "minimal-optimal" stopping point. What does "all-relevant" mean? The paper defines a variable as being relevant if there is a subset of attributes in the dataset among which the variable is not redundant when used for prediction. To illustrate this, suppose that we have two variables, A and B. These variables are correlated with one another, and correlated with the objective variable. However, A is a stronger signal than B is. In a regular decision tree the A feature will be given high importance whle the B feature will be mostly ignored in A's favor. If we were minimizing our variable count then we would remove B.

###### Decision trees, random forests, and gini inefficiency

The core algorithm behind boruta is **random forests**. Random forests are themselves based on **decision trees**. 

A decision tree is a sequence of steps (decisions, or splits) that are calcuated at training time. A simple decision tree might classify every point that has `X < 0.5` to class 0, and every point that has `X > 0.5` to class 1. Each new data point flows down the tree, either down the left branch or the right branch, and ultimately arrives at its classification result.

In practice decision trees can subject points to dozens of tests as they "flow" down the tree before assigning their ultimate classification, and if you allow arbitrarily many splits, you can generate a perfectly overfitted decision tree! To control for this we may specify that we will only continue to split nodes so long as the decision results in a certain minimum benefit. The definition of benefit that is usually used is reduction in Gini impurity. **Gini impurity** is an only slightly complicated measurement related to the Gini coefficient (and the subject of a future notebook) that essentially measures how well the classifier is performing on a given subset of data.

Gini impurity has the important property that it, like decision trees, is non-parametric, and hence (for a large enough sample size) we can expect it to work with data in any numerical format. This is the reason that decision trees in `sklearn` are able to return a `feature_importance_` attribute, and the reason that said attribute is so useful for choosing your input variables.

A random forest meanwhile is an ensemble of weak decision trees. In a random forest you train hundreds of purposefully overfitted decision trees, with each decision tree only gaining access to a random subset of the columns in the dataset. To classify an incoming point you have each of these trees cast a vote as to where to put it, and the majority vote wins.

The key insight in random forests, and the reason that they perform better than decision trees alone, is mass voting. By increasing the randomness of the decision trees being built we naturally increase their bias, but by then averaging their decisions we naturally reduce the variance. If a random forest is able to decrease variance more than it increases bias, relative to a single well-pruned decision tree, then it will perform better as a classifier on the dataset (albeit one that takes at least an order of magnitude longer to train).

**The algorithm**

Boruta works by taking this randomness even further. Here's the algorithm:

1. Extend the information system by adding copies of all variables (the information system is always extended by at least 5 shadow attributes, even if the number of attributes in the original set is lower than 5).
2. Shuffle the added attributes to remove their correlations with the response.
3. Run a random forest classifier on the extended information system and gather the Z scores computed.
4. Find the maximum Z score among shadow attributes (MZSA), and then assign a hit to every attribute that scored better than MZSA.
5. For each attribute with undetermined importance perform a two-sided test of equality with the MZSA.
6. Deem the attributes which have importance significantly lower than MZSA as ‘unimportant’ and permanently remove them from the information system.
7. Deem the attributes which have importance significantly higher than MZSA as ‘important’.
8. Remove all shadow attributes.
9. Repeat the procedure until the importance is assigned for all the attributes, or the algorithm has reached the previously set limit of the random forest runs.

What basically happens is that randomly shuffled shadow attributes are defined to establish a baseline performance for prediction against the target variable. Then a hypothesis test is used to determine, with a certain level of risk (0.05 by default), if each variable is correlated only randomly. Variables that fail to be rejected are removed from consideration.

Recall that we had a feature B that was redundant to feature A, but still related to the objective variable. By training a random forest boruta will, by sheer chance, generate decision trees having the B feature but lacking the A feature. In these cases, in the absence of feature A, feature B will be given high importance instead. This importance will bouy the average importance assigned to that variable and prevent it from failing the test and being removed.

Obviously the larger the feature space and the more variables B is redundant to, the larger the number of trees that must be in the forest for a tree where B is not redudant to occur. However, if you use a forest with a parameterization that has been cross-validated to be optimal, you should be fine.

#### Practical implementation of the Boruta algorithm to select feature importance

Here is a sample demonstration of the algorithm on a dataset obtained from [Kaggle](). 

In [29]:
df_X = df.iloc[:, 1:]
df_y = df.iloc[:, 0]

df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,39,7,77516,9,13,4,1,1,4,1,2174,0,40,39,0
1,50,6,83311,9,13,2,4,0,4,1,0,0,13,39,0
2,38,4,215646,11,9,0,6,1,4,1,0,0,40,39,0
3,53,4,234721,1,7,2,6,0,2,1,0,0,40,39,0
4,28,4,338409,9,13,2,10,5,2,0,0,0,40,5,0


In [30]:
clf = RandomForestClassifier(n_estimators=50, n_jobs=-1, max_depth=5)

In [31]:
trans = BorutaPy(clf, random_state=17, verbose=2)

In [33]:
sel = trans.fit_transform(df_X.values, df_y.values)

Iteration: 	1 / 100
Confirmed: 	0
Tentative: 	14
Rejected: 	0
Iteration: 	2 / 100
Confirmed: 	0
Tentative: 	14
Rejected: 	0
Iteration: 	3 / 100
Confirmed: 	0
Tentative: 	14
Rejected: 	0
Iteration: 	4 / 100
Confirmed: 	0
Tentative: 	14
Rejected: 	0
Iteration: 	5 / 100
Confirmed: 	0
Tentative: 	14
Rejected: 	0
Iteration: 	6 / 100
Confirmed: 	0
Tentative: 	14
Rejected: 	0
Iteration: 	7 / 100
Confirmed: 	0
Tentative: 	14
Rejected: 	0
Iteration: 	8 / 100
Confirmed: 	9
Tentative: 	1
Rejected: 	4


C:\Users\Ashoo\Miniconda3\lib\site-packages\boruta\boruta_py.py:418: RuntimeWarning: invalid value encountered in greater
  hits = np.where(cur_imp[0] > imp_sha_max)[0]


Iteration: 	9 / 100
Confirmed: 	9
Tentative: 	1
Rejected: 	4


C:\Users\Ashoo\Miniconda3\lib\site-packages\boruta\boruta_py.py:418: RuntimeWarning: invalid value encountered in greater
  hits = np.where(cur_imp[0] > imp_sha_max)[0]


Iteration: 	10 / 100
Confirmed: 	9
Tentative: 	1
Rejected: 	4


...

C:\Users\Ashoo\Miniconda3\lib\site-packages\boruta\boruta_py.py:418: RuntimeWarning: invalid value encountered in greater
  hits = np.where(cur_imp[0] > imp_sha_max)[0]


Iteration: 	14 / 100
Confirmed: 	9
Tentative: 	1
Rejected: 	4


C:\Users\Ashoo\Miniconda3\lib\site-packages\boruta\boruta_py.py:418: RuntimeWarning: invalid value encountered in greater
  hits = np.where(cur_imp[0] > imp_sha_max)[0]


Iteration: 	15 / 100
Confirmed: 	9
Tentative: 	1
Rejected: 	4
Iteration: 	16 / 100
Confirmed: 	9
Tentative: 	0
Rejected: 	5


BorutaPy finished running.

Iteration: 	17 / 100
Confirmed: 	9
Tentative: 	0
Rejected: 	5


C:\Users\Ashoo\Miniconda3\lib\site-packages\boruta\boruta_py.py:418: RuntimeWarning: invalid value encountered in greater
  hits = np.where(cur_imp[0] > imp_sha_max)[0]


In [34]:
boruta_selector = sel

In [37]:
#print ('\n Initial features: ', df.drop(['id','target'], axis=1).columns.tolist() )

# number of selected features
print ('\n Number of selected features:')
print (boruta_selector)


 Number of selected features:
[[     7  77516      9 ...      1     40      0]
 [     6  83311      9 ...      0     13      0]
 [     4 215646     11 ...      1     40      0]
 ...
 [     4 151910     11 ...      4     40      0]
 [     4 201490     11 ...      3     20      0]
 [     5 287927     11 ...      5     40      1]]


In [39]:
# check ranking of features
print ('\n Feature ranking:')
print (boruta_selector.ranking_)


 Feature ranking:


AttributeError: 'numpy.ndarray' object has no attribute 'ranking_'